# Ayudantía 6: Visualización

Altair es una librería de visualización estadistica para python, durante esta ayudantía haremos un repaso rapido de esta librería. Quiero notar que la gran mayoría de cosas hechas en esta se pueden replicar con `matplotlib`, y que la actividad indicada más abajo puede ser hecha de ambar formas.

Altair ofrece un gramática de visualización poderosa y concisa que permite construir un amplio rango de vizualización rápidas. Comenzaremos con un ejemplo rápido:

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt

from vega_datasets import data

pd.set_option('display.max_columns', 999)
alt.themes.enable('opaque')

%matplotlib inline

In [2]:
iris = data.iris()

alt.Chart(iris).mark_point().encode(
    x='petalLength',
    y='petalWidth',
    color='species'
).interactive()

alt.Chart(...)

Idealmente, esto debería mostrar un diagrama de dispersión de la longitud de los pétalos de diferentes especies de flores de Iris.

Ahora estamos listos para comenzar con nuestro propio gráfico.

Como recordatorio, aquí está la visualización de datos original que usaremos para trabajar:

![alt text](img/infographic.png "infografía original")



#### Análisis de datos exploratorios: 
A menudo, la visualización de datos es una parte clave del análisis de datos exploratorios, donde se encuentra un nuevo conjunto de datos y aún no sabe lo que contiene. Por ejemplo, los datos de un nuevo telescopio pueden contener efectos sistemáticos que conduzcan a datos de aspecto extraño. La visualización de los conjuntos de datos le ayuda a descubrir cómo se ven sus datos y qué sesgos pueden haber en ellos.
#### Explicar un resultado con una visualización: 
En nuestras publicaciones científicas (¡o no científicas!), A menudo usamos visualizaciones para explicar un resultado científico. En estos casos, ya conocemos la historia, nuestro resultado científico, por lo que en este caso nuestra tarea es asegurarnos de que nuestra visualización (1) represente los datos con precisión (todo lo demás estaría mintiendo), y (2) que permite la visor para comprender sus resultados y cómo ha llegado allí.
Obteniendo los datos
Al igual que con el tutorial, primero necesitamos obtener los datos en un formato legible por máquina de la figura.

Ahora, almacenemos nuestros datos en `pandas.DataFrame`:

In [3]:
ad_type = ["SEO", "Email", "PPC",  "PR", "Direct Mail", "OMB"]
roi = [68.9, 56.7, 52.4, 48.5, 37.4,  19.9]

data = {"adtype":ad_type, "roi":roi}

Aquí hemos creado dos listas, una con el tipo de publicidad, la otra con los valores de retorno de la inversión y luego pasamos ambas a un diccionario. Ahora podemos almacenarlos en un DataFrame:

In [4]:
df = pd.DataFrame(data)
df

,adtype,roi
0,SEO,68.9
1,Email,56.7
2,PPC,52.4
3,PR,48.5
4,Direct Mail,37.4
5,OMB,19.9


Altair puede trabajar con DataFrames, pero también puede guardar sus datos en un archivo json o un archivo csv (valores separados por comas), y luego pasarle el nombre del archivo y la ubicación (o URL) del archivo. Pandas pueden escribir ambos, lo que a veces puede ayudar a mantener más pequeño el tamaño de su cuaderno.

Pero en este caso, usemos el DataFrame.

### Gráficos *(de barra)* en Altair
Todas las figuras que generas en Altair siguen una convención similar, y todas son objetos de la clase `Chart()`. `Chart()` generalmente toma sus datos como entrada y luego usará métodos (funciones que se aplican a una clase específica de objeto) para especificar lo que realmente desea trazar.

Intentemos esto:

In [5]:
alt.Chart(df).mark_bar().encode(
    x = "adtype:O",
    y = "roi:Q"
)

alt.Chart(...)

¿Qué hemos hecho? La sintaxis puede parecer un poco rara, pero básicamente hemos llamado a varios métodos en la clase `Chart()` para decirle qué hacer. Una cosa que hemos usado es el método `mark_bar()`. Esto le dice a Altair que el gráfico que debe producir es un gráfico de barras. Luego, usamos el método de codificación `encode()` para decirle realmente qué valores trazar.

Prácticamente siempre tiene que llamar al método de codificación en algún lugar: podría pasar un *DataFrame* con muchas más columnas (por ejemplo, una columna llamada "costo"), y luego Altair no sabría cuáles usar y cuáles omitir . En el caso anterior, le hemos dicho que ponga `adtype` en el eje `x` y `roi` en el eje `y`. La sintaxis: `O` y: `Q` después de cada uno le dice a Altair que `adtype` contiene datos ordinales (es decir, categorías distintas y separadas), y `roi` contiene datos cuantitativos (es decir, números continuos).

También puede notar que Altair ha tomado algunas decisiones predeterminadas sobre las dimensiones del gráfico, las unidades en el eje y, las líneas de la cuadrícula en el fondo, etc. También agrega automáticamente etiquetas de eje cuando tiene una idea de qué etiquetas darle (eligió las etiquetas de columna de nuestro DataFrame en este caso).

Ese gráfico se ve muy bien, pero está un poco aplastado. Podemos deshacerlo configurando el tamaño del gráfico:

In [6]:
alt.Chart(df).mark_bar().encode(
    x = "adtype:O",
    y = "roi:Q"
).properties(
    width=300,
    height=200
)

alt.Chart(...)

Se ve mejor, ¿no? A diferencia de Seaborn, Altair automáticamente da a las barras el mismo color. Puede especificar el color de dos formas diferentes. Puede pasarlo directamente al método `mark_bar()`:

In [7]:
alt.Chart(df).mark_bar(color="darkblue").encode(
    x = "adtype:O",
    y = "roi:Q"
).properties(
    width=300,
    height=200
)

alt.Chart(...)

Cualquier código de color html funcionará, junto con cualquier código de color hexadecimal que pueda generar en ese sitio web u otros.

La otra cosa interesante que puede hacer es hacer algo llamado formato condicional. Supongamos que queremos colorear los métodos de publicidad online y los métodos de publicidad offline por separado. Agreguemos una columna a nuestro DataFrame para hacer exactamente eso:

In [8]:
df["online"] = ["yes", "yes", "yes", "no", "no", "yes"]
df

,adtype,roi,online
0,SEO,68.9,yes
1,Email,56.7,yes
2,PPC,52.4,yes
3,PR,48.5,no
4,Direct Mail,37.4,no
5,OMB,19.9,yes


Le dije que todos excepto *PR* y *Direct Mail* son versiones en línea. En lugar de pasar "color" al método `mark_bar()`, también podemos pasarlo al método `encoding()`. Veamos qué sucede cuando hacemos eso:

In [9]:
alt.Chart(df).mark_bar().encode(
    x = "adtype:O",
    y = "roi:Q",
    color= "online:O"
).properties(
    width=300,
    height=200
)

alt.Chart(...)

¡Ahora ha coloreado las dos barras de manera diferente y ha agregado automáticamente una leyenda!

Hay otros valores que puede elegir. Por ejemplo, en un diagrama de dispersión (*scatter plot*), puede usar el tamaño como canal de codificación para variar el tamaño de los puntos en función de alguna propiedad de cada punto. También puede codificar la opacidad, es decir, qué tan transparente es un elemento de la trama:

In [10]:
alt.Chart(df).mark_bar().encode(
    x = "adtype:O",
    y = "roi:Q",
    opacity= "online:O"
).properties(
    width=300,
    height=200
)

alt.Chart(...)

Aquí, el color sigue siendo el mismo, pero los métodos *offline* son menos opacos que los métodos *online*. Para obtener una lista de todas las diferentes codificaciones posibles y cómo usarlas, puede consultar la [documentación de Altair](https://altair-viz.github.io/user_guide/encoding.html) correspondiente.

Advertencia: ¡Poder codificar muchas propiedades diferentes en la misma gráfica no significa que sea una buena idea hacerlo! Es poco probable que las personas comprendan realmente más de 2-3 dimensiones diferentes en una trama. Siempre que sea posible, intente utilizar diferentes codificaciones para reforzar las propiedades importantes de los datos (por ejemplo, podría utilizar una codificación de "color" y "tamaño" utilizando la misma propiedad de datos, de modo que, por ejemplo, en su diagrama de dispersión también aparecerán marcas más grandes azul, y las marcas más pequeñas también serán verdes). Esto ayuda a los espectadores a comprender mejor la estructura de sus datos.

Puede que no nos gusten los valores predeterminados de Altair para el color y la opacidad, así que cambiémoslo. Podemos hacer eso usando la función condición así:

In [11]:
alt.Chart(df).mark_bar().encode(
    x = "adtype:O",
    y = "roi:Q",
    color= alt.condition(
        alt.datum.online == "yes", 
        alt.value("orange"),
        alt.value("steelblue"))
).properties(
    width=300,
    height=200
)

alt.Chart(...)

Aquí, le hemos dicho que grafique los métodos en línea en naranja y luego los métodos fuera de línea en azul. la función `condition()` toma como primer argumento una condición, aquí `alt.datum.online == "yes"`, que básicamente dice "tomar los puntos de datos en la columna `"online"` y encontrar todas las filas para las que el valor es `"yes"`. Los siguientes dos argumentos especifican qué debe hacer si esta condición es verdadera (aquí, use un color naranja) y qué debe hacer si la condición es falsa (use un color azul). La función de condición es bastante poderosa y útil en Altair, y se usa a menudo en visualizaciones interactivas, por lo que vale la pena comprender cómo funciona. Puede encontrar más información en la sección Interacción de la documentación.

### Especificación de ejes 
Lo que actualmente no me gusta de nuestro gráfico son las etiquetas de los ejes. Los nombres de las columnas que proporcionamos son bastante descriptivos y cortos, muy útiles cuando tienes que escribirlos muchas veces en un análisis de datos, pero si los incluyes en tu artículo, pocas personas entenderían lo que significan.

Démosle a nuestra trama algunas etiquetas de eje más descriptivas. Para ello, tendrá que saber que la sintaxis de codificación que usamos anteriormente, donde escribimos `x = "adtype: Q"` es una abreviatura de un comando más largo. La abreviatura es útil para gráficos rápidos en los que no le importa nada más que los valores predeterminados, pero cuando desee especificar más detalles, es posible que desee utilizar una sintaxis un poco más detallada:

In [12]:
alt.Chart(df).mark_bar().encode(
    x = alt.X("adtype:O", title="Type of Advertising"),
    y = alt.Y("roi:Q", title="Return on Investment [%]"),
    color= alt.condition(
        alt.datum.online == "yes", 
        alt.value("orange"),
        alt.value("steelblue"))
).properties(
    width=300,
    height=200
)

alt.Chart(...)

Aquí, hemos utilizado las funciones `alt.X()` y `alt.Y()` para especificar más detalles de nuestros ejes x e y, en este caso dándole la palabra clave title con títulos más descriptivos.

### Agregar texto
Usando una combinación de la función `condition()` que hemos visto antes y la propiedad `mark_text` podemos modificar el texto que se muestra en pantalla.

Para ello, también debe saber que es posible superponer gráficos uno encima del otro. En nuestro caso, vamos a colocar un elemento mark_bar y un elemento mark_text uno encima del otro en el mismo gráfico.

Para hacer esto, vamos a guardar nuestras marcas en variables. Hasta ahora, hemos escrito los comandos directamente en el campo de comando y el cuaderno ha generado automáticamente el resultado. Sin embargo, también podemos guardarlo en una variable del nombre que elijamos, y luego hacer que el cuaderno lo renderice más tarde cuando lo necesitemos.

Entonces, primero, generemos nuestro antiguo gráfico de barras y resaltemos "Correo electrónico" usando nuestro condicional:

In [13]:
roi_bars = alt.Chart(df).mark_bar().encode(
    x = alt.X("adtype:O", title="Type of Advertising"),
    y = alt.Y("roi:Q", title="Return on Investment [%]"),
    color= alt.condition(
        alt.datum.adtype == "Email", 
        alt.value("darkred"),
        alt.value("lightgrey"))
).properties(
    width=300,
    height=200
)

La ejecución de esa celda no trazó nada, porque hasta ahora, solo hemos guardado la especificación del gráfico en una variable, sin decirle al cuaderno que realmente lo renderice.

Usemos ahora `mark_text` para dibujar nuestros números:

In [14]:
roi_text = roi_bars.mark_text(
    align='center',
    baseline='middle',
    dy=-10,  # Mueve el texto hacia arriba para que no aparezca en la parte superior de la barra.
    color="darkred"
).encode(
    text="roi:Q",
    opacity= alt.condition(
        alt.datum.adtype == "Email", 
        alt.value(1.0),
        alt.value(0.0))
)
roi_bars + roi_text

alt.LayerChart(...)

¡Eso casi se parece a nuestra trama de matplotlib!

Como último paso, nos gustaría ordenar el gráfico de barras por altura en orden descendente. Podemos usar esto usando la propiedad "sort" en alt.Y:

In [15]:
roi_bars = (alt.Chart(df)
               .mark_bar()
               .encode(x = alt.X("adtype:O"
                                 , title = "Type of Advertising"
                                 , sort = alt.EncodingSortField(field = "roi",  # El campo que se utilizará para la clasificación.
                                                                order = "descending"  # El orden para clasificar
                                                               )

                                )
                       , y = alt.Y("roi:Q"
                                   , title = "Return on Investment [%]"
                                  )
                       , color = alt.condition(alt.datum.adtype == "Email"
                                               , alt.value("darkred")
                                               , alt.value("lightgrey")
                                             )
                    )
            .properties(width = 300
                        , height = 200
                       )
        )
roi_text = (roi_bars.mark_text(align = 'center'
                               , baseline = 'middle'
                               , dy = -10 # Mueve el texto hacia arriba para que no aparezca en la parte superior de la barra
                               , color = "darkred")
                    .encode(text = "roi:Q"
                            , opacity= alt.condition(alt.datum.adtype == "Email"
                                                     , alt.value(1.0)
                                                     , alt.value(0.0)
                                                    )
                           )
           )
roi_bars + roi_text

alt.LayerChart(...)

Muy bien, eso se parece bastante a una versión de matplotlib.

Sin embargo, una de las mejores cosas de altair es que es bastante fácil incluir interactividad. Para una funcionalidad de zoom simple, todo lo que necesita hacer es agregar `.interactive()`:

In [16]:
(roi_bars + roi_text).interactive()

alt.LayerChart(...)

Por supuesto, hacer zoom en un gráfico de barras no es particularmente satisfactorio, pero ya veremos el poder que nos puede dar es

Ahora vamos a hacer dos gráficos de barras y luego vincularlos:

In [17]:
costs = [1.2, 0.4, 1.5, 2.2, 8.7, 0.3]

df["costs"] = costs

In [18]:
base_chart = alt.Chart(df).mark_bar().encode(
    x = alt.X("adtype")
).properties(
    width=300,
    height=200
)

base_chart.encode(y=alt.Y("roi")) | base_chart.encode(y=alt.Y("costs"))

alt.HConcatChart(...)

Hacer gráficos de varios paneles es tan simple como hacer algo como `Chart1 | Chart2`: el | El símbolo le dice a Altair que debe hacer un diagrama de dos paneles. Para un gráfico de varios paneles apilado verticalmente, puede utilizar el símbolo &.

Ahora queremos agregar nuestras propiedades y selecciones nuevamente, y queremos asegurarnos de que se seleccionen las mismas en ambos lados:

In [19]:
selector = alt.selection(type="single", empty='none', on='click')

base_chart = alt.Chart(df).mark_bar().encode(
    x = alt.X("adtype:O", title="Type of Advertising",
               sort = alt.EncodingSortField(
                    field="roi",
                    order="descending"
        )
),
    color= alt.condition(
        selector, 
        alt.value("darkred"),
        alt.value("lightgrey")),
        tooltip=["roi", "costs", "online"] # display ROI and online properties on mouseover
).add_selection(
    selector
).properties(
    width=300,
    height=200
)


base_chart.encode(y = alt.Y("roi:Q", title="Return on Investment [%]")) | base_chart.encode(y = alt.Y("costs:Q", title="Cost in Million USD"))

alt.HConcatChart(...)

Ahora, cuando haga clic en cualquiera de las barras a cada lado, se resaltará la barra correspondiente en el otro lado. Resulta bastante fácil ver que el correo directo es una idea terrible (muy costosa, porque en realidad tienes que producir y enviar cartas físicas), y que el SEO y el correo electrónico son baratos y efectivos (por supuesto, acabamos de inventar los datos).


### Puntos clave
* altair es una poderosa biblioteca para generar visualizaciones (interactivas)

* Hacer coincidir el tipo de visualización con su tipo de datos puede mejorar drásticamente la legibilidad

* La elección de una paleta de colores informativa y de alto contraste puede ayudar a que la figura sea visible para una amplia gama de espectadores.

## Actividad: Índices de Costos de Vida

Estos índices están ajustados a la Ciudad de Nueva York (NYC). Lo que significa que para la Ciudad de Nueva York, cada índice debería marcar 100(%). Si otra ciudad tiene, por ejemplo, un índice de alquiler de 120, significa que en esa ciudad se paga de media por el alquiler un 20% más que en Nueva York. Si una ciudad tiene un índice de alquiler de 70, significa que en esa ciudad los alquileres son de media un 30% más baratos que en Nueva York.

* El Índice de Costo de Vida (Sin Alquiler) es un indicador relativo de los precios de bienes de consumo, incluyendo comestibles, restaurantes, transporte y servicios. El Índice de Costo de Vida no incluye gastos de residencia como alquileres o hipotecas. Si una ciudad tiene un Costo de Vida de 120, significa que Numbeo estima que es un 20% más cara que Nueva York (sin contar alquiler).

* El Índice de Alquiler es una estimación de precios de alquiler de apartamentos de una ciudad comparada con Nueva York. Si el Índice de Alquiler es 80, Numbeo estima que el precio de los alquileres en esa ciudad es de media un 20% más barato que en Nueva York.

* El Índice de Comestibles es una estimación de los precios de la compra de una ciudad en comparación con Nueva York. Para calcular esta sección, Numbeo utiliza el peso de los artículos en la sección "Mercados" por cada ciudad.

* El Índice de Restaurantes es una comparación de precios de comidas y bebidas en bares y restaurantes en comparación con NY.

* El Índice de Costo de Vida más Alquiler es una estimación de precios de consumo incluyendo alquiler en comparación con la Ciudad de Nueva York.

* El Poder Adquisitivo Local muestra la capacidad adquisitiva relativa a la hora de comprar bienes y servicios en una ciudad determinada, con relación al salario medio de la ciudad. Si el poder adquisitivo doméstico es 40, significa que los habitantes de dicha ciudad con salario medio pueden permitirse comprar una media de 60% menos bienes y servicios que los habitantes de Nueva York con salario medio. 

Para más información sobre los pesos utilizados (fórmula completa) puedes visitar: [motivación y metodología](https://es.numbeo.com/coste-de-vida/motivaci%C3%B3n-y-metodolog%C3%ADa).

Para comenzar es necesario instalar el paquete `lxml` en tu entorno virtual de conda para poder descargar los datos. Basta con ejecutar 

`conda install -n mat281 lxml`

O cambia `mat281` por el ambiente que estés utilizando.

Se disponibiliza a continuación la carga de datos de un dataframe.

In [ ]:
years = [2015, 2016, 2017, 2018, 2019, 2020]
life_cost = (
    pd.concat(
        {
            year: (
                pd.read_html(f"https://www.numbeo.com/cost-of-living/rankings.jsp?title={year}")[1]
                .rename(columns=lambda x: x.lower().replace(" ", "_"))
                .assign(rank=lambda x: x.index + 1)
                .set_index("rank")
            ) for year in years
        }
    )
    .rename_axis(["year", "rank"])
    .reset_index()
)
life_cost.head()

### Ejercicio 1.1 

Explique lo que se hizo en la celda anterior detalladamente.

_## TU RESPUESTA AQUÍ ##_

### Ejercicio 1.2 (10 pts)

Genera un histograma del índice del costo de vida (sin alquiler) para cada año (es decir, 6 histogramas).


In [ ]:
## FIX ME PLEASE ##

_## TU RESPUESTA AQUÍ ##_

### *spoiler*
Presione los tres puntitos si quiere un outline de como hacer el gráfico

In [ ]:
(alt.Chart(life_cost)
    .mark_bar()
    .encode(x = alt.X( #fill
                      title = #fill, 
                      sort = alt.EncodingSortField(field = #fill, 
                                                   order = #fill
                                                  ), 
                      axis = alt.Axis(labelFontSize = 13)
                     ),
            y = alt.Y(#fill, 
                      title = #fill
                     ),
            row = alt.Row(#fill, 
                          title = #fill, 
                          header = alt.Header(labelFontSize=15)
                         ),
            tooltip = #fill
           )
    .properties(title = alt.TitleParams(text = #fill, 
                                        align = #fill, 
                                        dy = #fill, 
                                        anchor = #fill
                                       )
               )
    .resolve_scale(x = #fill)
)

### Ejercicio 1.3 ( pts)

Grafica el índice de restaurantes a través de los años para diez ciudades escogidas pseudo-aleatoriamente (variable `my_cities` de la celda siguiente) en un mismo gráfico. Recuerda escoger el tipo de gráfico adecuadamente.

¿Ves alguna relación? ¿Qué podrías decir del gráfico? ¿Por qué no graficar todas las ciudades en lugar de solo escoger algunas?

In [ ]:
seed = 420  #para reproduicibilidad
my_cities = life_cost["city"].drop_duplicates().sample(n=10, random_state=seed).values

In [ ]:
## FIX ME PLEASE ##

_## TU RESPUESTA AQUÍ ##_

### *spoiler*
Presione los tres puntitos si quiere un outline de como hacer el gráfico

In [ ]:
(alt.Chart(life_cost[lambda #fill])
    .mark_line()
    .encode(alt.X(#fill, 
                  title = #fill, 
                  axis = alt.Axis(labelFontSize=11)
                 ),
            alt.Y(#fill, 
                  title = #fill
                 ),
            alt.Color(#fill, 
                      title = #fill
                     ),
            tooltip = #fill
           )
    .properties(title = alt.TitleParams(text =#fill, 
                                        anchor = #fill, 
                                        dy = #fill, 
                                        dx = #fill
                                       )
               )
)

### Ejercicio 1.4

Genera un mapa de calor tal que:

- El eje horizontal corresponda a cada uno de los índices.
- El eje vertical corresponda a cada una de las ciudades de `my_cities`.
- El color y valor en cada celda sea el promedio de los indicadores.
    - El valor de la celda debe tener solo dos decimales.

In [ ]:
redable_index_names = {
    'cost_of_living_index': 'Costo de Vida',
    'rent_index': 'Alquiler',
    'cost_of_living_plus_rent_index': 'Costo de Vida + Alquiler',
    'groceries_index': 'Comestibles',
    'restaurant_price_index': 'Restaurantes',
    'local_purchasing_power_index': 'Poder Adquisitivo Local'
}

In [ ]:
## FIX ME PLEASE ##

_## TU RESPUESTA AQUÍ ##_

### *spoiler*
Presione los tres puntitos si quiere un outline de como hacer el gráfico

In [ ]:
df = (life_cost[lambda df: df['city'].isin(#fill)]
                                     .drop(#fill)
                                     .groupby(#fill)
                                     .agg(#fill)
                                     .rename(#fill)
                                     .round(#fill)
                                     .reset_index()
                                     .melt(#fill)

    )

In [ ]:
base = (alt.Chart(df)
           .encode(x = alt.X(#fill, 
                             title = #fill, 
                             axis = alt.Axis(orient = #fill, 
                                             labelAngle = #fill, 
                                             labelFontSize=#fill
                                            )
                            ),
                   y=alt.Y(#fill, 
                           title = #fill, 
                           axis = alt.Axis(#fill)
                          )
                  )
           .properties(title = alt.TitleParams(text =#fill, 
                                               dy = #fill, 
                                               anchor = #fill, 
                                               dx = #fill
                                             ),
                       width = #fill,
                       height = #fill
                      )
     )
heatmap = (base.mark_rect()
               .encode(color = alt.Color(#fill,
                                         scale = alt.Scale(scheme = #fill),
                                         title=#fill,
                                        ),     
                       tooltip = [alt.Tooltip(#fill),
                                  alt.Tooltip(#fill),
                                  alt.Tooltip(#fill)
                                 ]
                       )
          )
text = (base.mark_text(#fill)
            .encode(text=#fill,
                    color=alt.condition(#fill,
                                        #fill,
                                        #fill
                                       )
                   )
       ) 
heatmap + text